# Red Wine Quality Project
***
***

# Goal
***
- Predict the quality of red wines based on some of their various physiochemical attributes

# Setup
***

In [1]:
# establishing environment
import sklearn
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE

import warnings
warnings.filterwarnings("ignore")

# Acquire
Acquiring the data for this project
***

In [2]:
wines = pd.read_csv('winequality-red.csv')

wines.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


# Prepare
Preparing the data for this project

## Splitting data

In [4]:
# splitting data
train_validate, test = train_test_split(wines, test_size=.2, random_state=123)
train, validate = train_test_split(train_validate, test_size=.3, random_state=123)

## Scaling data

In [27]:
# identifying columns to scale
col_to_scale = []

for x in wines:
    if (wines[x].min() < 0 or wines[x].max() > 1) and x != 'quality':
        col_to_scale.append(x)

col_to_scale

['fixed acidity',
 'volatile acidity',
 'residual sugar',
 'free sulfur dioxide',
 'total sulfur dioxide',
 'density',
 'pH',
 'sulphates',
 'alcohol']

In [23]:
wines['pH'].min(), wines['pH'].max()

(2.74, 4.01)

In [3]:
# creating scaler object
scaler = sklearn.preprocessing.MinMaxScaler()

# columns to scale
col_to_scale = ['age', 'bmi', 'children']
col_scaled_names = ['age_s', 'bmi_s', 'children_s']

# creating columns for scaled values
train['age_s'], train['bmi_s'], train['children_s'] = 0,0,0
validate['age_s'], validate['bmi_s'], validate['children_s'] = 0,0,0
test['age_s'], test['bmi_s'], test['children_s'] = 0,0,0

# fitting scaler to train column and scaling after
train[col_scaled_names] = scaler.fit_transform(train[col_to_scale])

# scaling data in validate and test dataframes
validate[col_scaled_names] = scaler.transform(validate[col_to_scale])
test[col_scaled_names] = scaler.transform(test[col_to_scale])

(1599, 12)